In [1]:
! pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
import shutil
from pyspark.sql.types import *
import pyspark.sql.functions as Spark_f
from pyspark import SparkConf

In [3]:
spark = SparkSession.builder.appName("PySparkAmazonReviewsJob").getOrCreate()

24/10/23 15:29:53 WARN Utils: Your hostname, NBM-WXX9-7572c0be resolves to a loopback address: 127.0.1.1; using 192.168.88.79 instead (on interface wlp1s0)
24/10/23 15:29:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/23 15:29:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
best_reviews = spark.read.parquet("/dataset/output/best_reviews")
best_reviews = best_reviews[best_reviews.helpful_vote!=18130]
best_reviews.show()

+-----------+------+--------------------+--------------------+--------------------+-------------+-----------------+------------+
|parent_asin|rating|               title|                text|             user_id|    timestamp|verified_purchase|helpful_vote|
+-----------+------+--------------------+--------------------+--------------------+-------------+-----------------+------------+
| 1501175564|   2.0|This pretty much ...|I'm a non-partisa...|AFENZBXVDFQDJADMN...|1505435675735|             true|       14835|
| 1501175564|   1.0|   Litany of Excuses|It's the same tir...|AGA7SSJTMKSLYTSGF...|1505508811827|             true|        7791|
| 1543024971|   5.0|A Must-Read for M...|A few years ago, ...|AGZ7B6YEHVZHQHZFN...|1488943350000|             true|        7775|
| 1501175564|   1.0|A complete waste ...|A complete waste ...|AG6I7QNRWQPS3UVZI...|1505365873977|             true|        7442|
| 1501175564|   1.0|"And I would've g...|Blamefest, but I ...|AHFMG76RZZDER4UOX...|1505354682473|

In [6]:
book_meta = spark.read.parquet("/dataset/output/book_metadata")
book_meta.show()

+-----------+-------------+--------------------+--------------+-------------+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+--------------------+
|parent_asin|main_category|               title|average_rating|rating_number|            features|         description|              images|videos|             details|first_category|     second_category|remaining_categories|
+-----------+-------------+--------------------+--------------+-------------+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+--------------------+
| 0001371029|        Books|Treasury of Nurse...|           4.5|           43|[Vintage children...|                  []|[{large -> https:...|    []|                  {}|         Books|    Children's Books|                  []|
| 0001486632|        Books|New England Chris...|           5.0|            1|                  [

In [7]:
book_meta = book_meta.select("parent_asin","title","description")
book_meta.show()
best_reviews = best_reviews.select("parent_asin",col("text").alias("review"))

+-----------+--------------------+--------------------+
|parent_asin|               title|         description|
+-----------+--------------------+--------------------+
| 0001371029|Treasury of Nurse...|                  []|
| 0001486632|New England Chris...|                  []|
| 0002151863|Success Secrets: ...|                  []|
| 0002192942|A Field Guide to ...|                  []|
| 0002554356|Polaroids from th...|                  []|
| 0004126742|How to Draw and P...|                  []|
| 0004127382|The East India Co...|                  []|
| 0006132685|Hitler, the last ...|                  []|
| 0006375731|Supreme Vegetaria...|                  []|
| 0006922627|      Emperor's Pony|                  []|
| 0007114346|      Wolf's Brother|                  []|
| 0007119313|Murder on the Ori...|                  []|
| 0007155387|        Side Effects|[Review, This col...|
| 0007176848|THE GENTLE BIRTH ...|                  []|
| 0007207492|Order In Chaos (T...|              

In [8]:
result_df = book_meta.join(best_reviews, "parent_asin")
result_df = result_df.groupBy(["parent_asin","title","description"]).agg(Spark_f.collect_list("review").alias("reviews"))
# result_df.filter(Spark_f.size("reviews")>2).select("parent_asin",result_df.reviews[0],result_df.reviews[1]).show(20,False)

In [9]:
result_df.coalesce(1).write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/output/result_df")

24/10/23 15:30:09 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
